In [19]:
import torch
import random

from sent_order.conll import Corpus, Document

from torch.nn import functional as F
from itertools import islice
from tqdm import tqdm_notebook

In [20]:
model = torch.load(
    '../../data/coref.4.bin',
    map_location={'cpu': 'cpu'},
)

In [22]:
model.eval()

Coref(
  (encode_doc): DocEncoder(
    (embeddings): WordEmbedding(39414, 300)
    (lstm): LSTM(300, 200, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (score_spans): SpanScorer(
    (attention): Scorer(
      (score): Sequential(
        (0): Linear(in_features=400, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
    (width_embeddings): DistanceEmbedding(9, 20)
    (sm): Scorer(
      (score): Sequential(
        (0): Linear(in_features=1120, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
  )
  (score_pairs): PairScorer(
    (dist_embeddings): DistanceEmbedding(9, 20)
    (sa): Scorer(
      (score): Sequential(
        (0): Linear(

In [23]:
c = Corpus.from_combined_file('../../data/test.conll')

In [24]:
def score(spans):
    return sum([sum(F.softmax(span.sij, 0)[:-1]) for span in spans])

In [25]:
correct, total = 0, 0
for sent_tokens in tqdm_notebook(islice(c.sent_chunk_tokens(), 200)):
    
    shuffled = sorted(sent_tokens, key=lambda x: random.random())
    
    d1 = Document([t for s in sent_tokens for t in s])
    d2 = Document([t for s in shuffled for t in s])
    
    try:
    
        spans1 = model(d1)
        spans2 = model(d2)
        
        if score(spans1) > score(spans2):
            correct += 1

        total += 1
        
    except:
        pass

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [26]:
correct / total

0.49246231155778897